In [6]:
import pandas as pd

# Load the dataset
df = pd.read_excel("Recommender_University_Data.xlsx")

# View first few rows
df.head()

,Student_ID,Interest_1,Interest_2,Parent_Choice,Peer_Choice,Final_Choice,University
0,S001,Law,Education,Computer Science,Engineering,Engineering,Ahmadu Bello University
1,S002,Computer Science,Engineering,Education,Engineering,Engineering,Ahmadu Bello University
2,S003,Business,Medicine,Psychology,Fine Arts,Medicine,UNILORIN
3,S004,Computer Science,Business,Psychology,Psychology,Psychology,UNN
4,S005,Education,Physics,Psychology,Engineering,Education,UNN


In [7]:
!pip install scikit-learn

In [8]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

In [9]:
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Student_ID     100 non-null    object
 1   Interest_1     100 non-null    object
 2   Interest_2     100 non-null    object
 3   Parent_Choice  100 non-null    object
 4   Peer_Choice    100 non-null    object
 5   Final_Choice   100 non-null    object
 6   University     100 non-null    object
dtypes: object(7)
memory usage: 5.6+ KB


,Student_ID,Interest_1,Interest_2,Parent_Choice,Peer_Choice,Final_Choice,University
0,S001,Law,Education,Computer Science,Engineering,Engineering,Ahmadu Bello University
1,S002,Computer Science,Engineering,Education,Engineering,Engineering,Ahmadu Bello University
2,S003,Business,Medicine,Psychology,Fine Arts,Medicine,UNILORIN
3,S004,Computer Science,Business,Psychology,Psychology,Psychology,UNN
4,S005,Education,Physics,Psychology,Engineering,Education,UNN


In [10]:
# Create a matrix of students and their final choices
student_program_matrix = pd.get_dummies(df.set_index('Student_ID')['Final_Choice'])

# Show the matrix
student_program_matrix.head()

,Art,Biochemistry,Biology,Business,Business Admin,Computer Science,Economics,Education,Engineering,Fine Arts,Law,Medicine,Physics,Psychology
Student_ID,,,,,,,,,,,,,,
S001,False,False,False,False,False,False,False,False,True,False,False,False,False,False
S002,False,False,False,False,False,False,False,False,True,False,False,False,False,False
S003,False,False,False,False,False,False,False,False,False,False,False,True,False,False
S004,False,False,False,False,False,False,False,False,False,False,False,False,False,True
S005,False,False,False,False,False,False,False,True,False,False,False,False,False,False


In [11]:
# Compute cosine similarity
similarity_matrix = cosine_similarity(student_program_matrix)

# Convert to DataFrame for readability
similarity_df = pd.DataFrame(similarity_matrix, index=student_program_matrix.index, columns=student_program_matrix.index)

# Show similarities for the first student
similarity_df.iloc[0].sort_values(ascending=False).head()

Student_ID
S001    1.0
S043    1.0
S078    1.0
S071    1.0
S066    1.0
Name: S001, dtype: float64

In [12]:
def recommend_program(student_id, top_n=3):
    if student_id not in similarity_df.index:
        return "Student not found."

    # Get similar students
    similar_scores = similarity_df[student_id].sort_values(ascending=False)[1:top_n+1]

    # Find what those similar students picked as Final_Choice
    similar_students = df[df['Student_ID'].isin(similar_scores.index)]
    
    recommended_programs = similar_students['Final_Choice'].value_counts().head(top_n).index.tolist()
    
    return recommended_programs

In [31]:
recommend_program("S003")  # Replace with an actual ID from your dataset

['Medicine']

In [26]:
recommend_program("S005")  # Replace with an actual ID from your dataset

['Education']

In [33]:
import pandas as pd
from dash import Dash, dcc, html, Input, Output
import plotly.express as px

In [34]:
app = Dash(__name__)

In [35]:
app.layout = html.Div([
    html.H1("AI-Powered University Recommender Dashboard"),

    html.Label("Select Student:"),
    dcc.Dropdown(
        id='student-dropdown',
        options=[{'label': sid, 'value': sid} for sid in df['Student_ID']],
        value=df['Student_ID'][0]
    ),

    html.Br(),

    html.Div(id='student-insight'),

    html.Br(),

    dcc.Graph(id='degree-bar'),

    html.Br(),

    dcc.Graph(id='university-bar')
])

In [36]:
@app.callback(
    [Output('student-insight', 'children'),
     Output('degree-bar', 'figure'),
     Output('university-bar', 'figure')],
    [Input('student-dropdown', 'value')]
)
def update_dashboard(student_id):
    student = df[df['Student_ID'] == student_id].iloc[0]

    insight = html.Div([
        html.H3(f"Choices for Student ID: {student_id}"),
        html.P(f"Interest 1: {student['Interest_1']}"),
        html.P(f"Interest 2: {student['Interest_2']}"),
        html.P(f"Parent's Choice: {student['Parent_Choice']}"),
        html.P(f"Peer's Choice: {student['Peer_Choice']}"),
        html.P(f"Final Choice: {student['Final_Choice']}"),
        html.P(f"University Chosen: {student['University']}")
    ])

    degree_fig = px.bar(
        df['Final_Choice'].value_counts().reset_index(),
        x='index', y='Final_Choice',
        labels={'index': 'Degree Program', 'Final_Choice': 'Number of Students'},
        title='Most Chosen Degree Programs'
    )

    university_fig = px.pie(
        df['University'].value_counts().reset_index(),
        names='index', values='University',
        title='University Preference Distribution'
    )

    return insight, degree_fig, university_fig

In [17]:
html.Div(id='ai-insight', style={'marginTop': '20px', 'fontWeight': 'bold', 'color': 'green'}),

(Div(id='ai-insight', style={'marginTop': '20px', 'fontWeight': 'bold', 'color': 'green'}),)

In [18]:
app.layout = html.Div([
    html.H1("University Admission Recommender Dashboard"),
    dcc.Graph(id='final-choice-distribution'),
    html.Div(id='ai-insight', style={'marginTop': '20px', 'fontWeight': 'bold', 'color': 'green'})
])

In [19]:
@app.callback(
    Output('final-choice-distribution', 'figure'),
    Output('ai-insight', 'children'),
    Input('final-choice-distribution', 'id')  # Dummy input just to trigger
)
def update_dashboard(_):
    fig = px.histogram(df, x='Final_Choice', color='University',
                       title='Distribution of Final Choices Across Universities')

    ai_insight = generate_ai_insight(df)
    return fig, ai_insight

In [1]:
pip install dash-bootstrap-components

In [5]:
import pandas as pd
from dash import Dash, dcc, html, Input, Output, State
import dash_bootstrap_components as dbc
from collections import Counter

# Load data
df = pd.read_excel("Recommender_University_Data.xlsx")

# Initialize Dash app
app = Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])
server = app.server

# App layout
app.layout = dbc.Container([
    html.H2("🎓 Smart University & Course Recommender", className="my-3 text-center text-primary"),

    dbc.Row([
        dbc.Col([
            dbc.Label("Interest 1"),
            dcc.Input(id="interest1", type="text", placeholder="e.g. Science", className="form-control mb-2"),

            dbc.Label("Interest 2"),
            dcc.Input(id="interest2", type="text", placeholder="e.g. Tech", className="form-control mb-2"),

            dbc.Label("Parent or Peer Choice"),
            dcc.Input(id="influence", type="text", placeholder="e.g. Medicine", className="form-control mb-3"),

            dbc.Button("Get Recommendations", id="recommend-btn", color="primary", className="mb-3", n_clicks=0),
        ], md=5),

        dbc.Col([
            html.H4("🎯 Recommended Degree Programs"),
            html.Div(id="recommended-courses", className="mb-4 text-success"),

            html.H4("🏫 Recommended Universities"),
            html.Div(id="recommended-universities", className="mb-4 text-success"),

            html.H4("💡 AI Explanation"),
            html.Div(id="ai-explanation", className="text-muted"),
        ], md=7),
    ])
], fluid=True)

# Callback
@app.callback(
    Output("recommended-courses", "children"),
    Output("recommended-universities", "children"),
    Output("ai-explanation", "children"),
    Input("recommend-btn", "n_clicks"),
    State("interest1", "value"),
    State("interest2", "value"),
    State("influence", "value")
)
def recommend(n_clicks, interest1, interest2, influence):
    if n_clicks == 0:
        return "", "", ""

    # Ensure lowercase matching
    interest1 = (interest1 or "").lower()
    interest2 = (interest2 or "").lower()
    influence = (influence or "").lower()

    # Prepare dataset for comparison
    temp_df = df.copy()
    temp_df["match_score"] = 0

    # Compare each row and count how many fields match
    for i, row in temp_df.iterrows():
        score = 0
        if row["Interest_1"].lower() == interest1:
            score += 1
        if row["Interest_2"].lower() == interest2:
            score += 1
        if row["Parent_Choice"].lower() == influence or row["Peer_Choice"].lower() == influence:
            score += 1
        temp_df.at[i, "match_score"] = score

    # Filter similar students
    similar = temp_df[temp_df["match_score"] >= 2]

    if similar.empty:
        return "No similar profiles found.", "", ""

    # Recommend top 3 programs
    top_programs = Counter(similar["Final_Choice"]).most_common(2)
    recommended_programs = [f"{i+1}. {p[0]}" for i, p in enumerate(top_programs)]

    # Recommend top 3 universities
    top_unis = Counter(similar["University"]).most_common(3)
    recommended_universities = [f"{i+1}. {u[0]}" for i, u in enumerate(top_unis)]

    # AI-style explanation
    explanation = (
        f"We found {len(similar)} students with similar interests or influence.\n"
        f"Most of them chose **{top_programs[0][0]}** as their degree program and preferred **{top_unis[0][0]}**."
    )

    return html.Ul([html.Li(prog) for prog in recommended_programs]), \
           html.Ul([html.Li(uni) for uni in recommended_universities]), \
           explanation

# Run app
if __name__ == '__main__':
    app.run(debug=True, port=8056)